In [19]:
import numpy as np
import os
from PIL import Image


def restore(sigma, u, v, K):  # 奇异值、左特征向量、右特征向量
    m = len(u)
    n = len(v[0])
    a = np.zeros((m, n))
    for k in range(K):
        uk = u[:, k].reshape(m, 1)
        vk = v[k].reshape(1, n)
        a += sigma[k] * np.dot(uk, vk)   # 前 k 个奇异值的加和
    a = a.clip(0, 255)
    return np.rint(a).astype('uint8')


if __name__ == "__main__":
    image_name="avatar3.png"
    A = Image.open(f"./data/{image_name}", 'r')
    output_path = './data'
    a = np.array(A)
    print('type(a) = ', type(a))
    print('原始图片大小：', a.shape)

   # 图片有RGB三原色组成，所以有三个矩阵
    u_r, sigma_r, v_r = np.linalg.svd(a[:, :, 0])    # 奇异值分解
    u_g, sigma_g, v_g = np.linalg.svd(a[:, :, 1])
    u_b, sigma_b, v_b = np.linalg.svd(a[:, :, 2])

   # 仅使用前1个，2个，...，50个奇异值的结果
    K = 80
    for k in range(1, K+1):
        R = restore(sigma_r, u_r, v_r, k)
        G = restore(sigma_g, u_g, v_g, k)
        B = restore(sigma_b, u_b, v_b, k)
        I = np.stack((R, G, B), axis=2)   # 将矩阵叠合在一起，生成图像
        Image.fromarray(I).save(f'{output_path}/output/{k}.jpg')


type(a) =  <class 'numpy.ndarray'>
原始图片大小： (1078, 1920, 3)
